In [1]:

import os

import openai
import pandas as pd

openai.api_key = "YOUR_API_KEY"

# Prepare Data

In [2]:
train_df = pd.read_csv('./data/train.csv')
val_df = pd.read_csv('./data/val.csv')


# We can create prompt in 2 ways: instruction or raw text
# For instruction propmpt we will use a human understandable textual instruction.
# For raw prompt we will use raw text with special separator between propmpt and completion

def create_instruction_prompt(text, all_labels):
    prompt =  f''' Classify the following messages into one of the following categories: {','.join(all_labels)}

Message: {text}

Category:'''
    return prompt

def create_raw_prompt(text):
    prompt =  f'''{text} /n/n###/n/n'''
    return prompt

# For classification task we need 1 token completion. The completion token must be in model vocabulary. 
# GPT tokenization required completion tokens started with whitespace.

train_df['completion'] = train_df['label'].apply(lambda x: ' '+ x)
val_df['completion'] = val_df['label'].apply(lambda x: ' ' + x)

# instruction based prompt
all_labels = set(train_df['completion'].unique())
train_df['prompt'] = train_df['text'].apply(lambda x: create_instruction_prompt(x, all_labels))
val_df['prompt'] = val_df['text'].apply(lambda x: create_instruction_prompt(x, all_labels))

# # raw text based prompt
# train_df['completion'] = train_df['text'].apply(create_raw_prompt)
# val_df['completion'] = val_df['text'].apply(create_raw_prompt)

train_df[['prompt', 'completion']].to_json("./data/train_hatespeech.jsonl", orient='records', lines=True)
val_df[['prompt', 'completion']].to_json("./data/val_hatespeech.jsonl", orient='records', lines=True)


# Finetune

In [ ]:
# Run in terminal
# Use OpenAI API key https://platform.openai.com/account/api-keys

# openai -k YOUR_API_KEY api fine_tunes.create -t ./data/train_hatespeech.jsonl -v ./data/val_hatespeech.jsonl -m ada --compute_classification_metrics --classification_n_classes 3 --n_epochs 4
# openai -k YOUR_API_KEY api fine_tunes.create -t ./data/train_hatespeech.jsonl -v ./data/val_hatespeech.jsonl -m davinci --compute_classification_metrics --classification_n_classes 3 --n_epochs 4
# openai wandb sync

# Finetune report

https://api.wandb.ai/links/vetka925/tgmn8w11

# Validation of zero-shot ChatGPT (gpt-3.5-turbo)

In [10]:
%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 238.3 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 0.26.5
    Uninstalling openai-0.26.5:
      Successfully uninstalled openai-0.26.5
Note: you may need to restart the kernel to use updated packages.


In [2]:
val_df = pd.read_csv('./data/val.csv')


def create_instruction_prompt(text, all_labels):
    prompt =  f''' Classify the following messages into one of the following categories: {','.join(all_labels)}

Message: {text}

Category:'''
    return prompt


all_labels = set(val_df['label'].unique())
val_df['prompt'] = val_df['text'].apply(lambda x: create_instruction_prompt(x, all_labels))



In [9]:
from tqdm.notebook import tqdm
import time

MODEL = "gpt-3.5-turbo"

result = []
for prompt in tqdm(val_df['prompt']):
    time.sleep(0.2)
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a hate speech, offensive language classifier."},
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )
    result.append(response['choices'][0]['message']['content'].lower())


  0%|          | 0/200 [00:00<?, ?it/s]

In [10]:
print(result[:10])

['hate', 'offensive', 'hate/offensive', 'offensive', 'neutral', 'neutral', 'offensive', 'offensive', 'offensive', 'offensive']


In [11]:
from sklearn.metrics import accuracy_score

print(f"Accuracy: {accuracy_score(result, val_df['label'])}")

Accuracy: 0.5
